In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16, InceptionV3, MobileNet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import os
os.chdir('/content/drive/MyDrive')

In [59]:
!unzip -qq textile_dataset1.zip

In [60]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   fill_mode='nearest')

In [61]:
train_generator = train_datagen.flow_from_directory('./data/train',
                                                    target_size=(150,150),
                                                    batch_size=5,
                                                    class_mode='binary')

Found 1445 images belonging to 2 classes.


In [62]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('./data/test',
                                                    target_size=(150,150),
                                                    batch_size=5,
                                                    class_mode='binary')


Found 870 images belonging to 2 classes.


In [64]:
transfer_models = [VGG16(weights='imagenet', include_top=False,input_shape=(150,150,3)), InceptionV3(weights='imagenet', include_top=False,input_shape=(150,150,3)), MobileNet(weights='imagenet', include_top=False,input_shape=(150,150,3))]
models = ['VGG16.hdf5', 'VGG16_2.hdf5', 'InceptionV3.hdf5', 'InceptionV3_2.hdf5', 'MobileNet.hdf5', 'MobileNet_2.hdf5']

model_index = 0
for transfer_model in transfer_models:
  transfer_model.trainable = False
  transfer_model.summary()

  for layer in range(1, 3):
    finetune_model = Sequential()
    finetune_model.add(transfer_model)
    finetune_model.add(Flatten())
    finetune_model.add(Dense(64 * layer, activation='relu'))
    finetune_model.add(Dense(1,activation='sigmoid'))
    finetune_model.summary()

    finetune_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

    modelpath="./model/" + models[model_index]
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', save_best_only=True, verbose=1)

    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

    history= finetune_model.fit(train_generator, epochs=100,
                      validation_data= test_generator,
                      validation_steps= len(test_generator),
                      callbacks=[early_stopping_callback,checkpointer])

    from tensorflow.keras.models import load_model
    bestmodel=load_model(modelpath)
    score = bestmodel.evaluate(test_generator, steps=len(test_generator))
    print('Test accuracy:', score[1])

    model_index += 1

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


289/289 [==============================] - 24s 79ms/step - loss: 0.4661 - accuracy: 0.8035 - val_loss: 0.3757 - val_accuracy: 0.8391
Epoch 2/100
289/289 [==============================] - ETA: 0s - loss: 0.4204 - accuracy: 0.8249
Epoch 2: val_loss improved from 0.37569 to 0.32374, saving model to ./model/VGG16.hdf5
289/289 [==============================] - 20s 70ms/step - loss: 0.4204 - accuracy: 0.8249 - val_loss: 0.3237 - val_accuracy: 0.8655
Epoch 3/100
289/289 [==============================] - ETA: 0s - loss: 0.3991 - accuracy: 0.8284
Epoch 3: val_loss improved from 0.32374 to 0.28964, saving model to ./model/VGG16.hdf5
289/289 [==============================] - 21s 71ms/step - loss: 0.3991 - accuracy: 0.8284 - val_loss: 0.2896 - val_accuracy: 0.8759
Epoch 4/100
288/289 [============================>.] - ETA: 0s - loss: 0.3867 - accuracy: 0.8493
Epoch 4: val_loss did not improve from 0.28964
289/289 [==============================] - 20s 70ms/step - loss: 0.3867 - accuracy: 0.849

In [ ]:
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()